# Recurrent Neural Networks

In many cases, when the order of the data carries significant meaning, eg: time series data, sound waves, natural language, its most useful to use Recurrent Neural Networks.

![Recurrent Neural Network Structure](res/rnn.png)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM # Use CuDNNLSTM if GPU available!

In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 32s 3us/step


In [3]:
print(X_train.shape)

(60000, 28, 28)
